In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split

In [2]:
# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# define transform
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor()
])

In [4]:
# set image paths
train_dir = '/kaggle/input/intel-image-classification/seg_train/seg_train'
full_dataset = datasets.ImageFolder(root = train_dir, transform = transform)

In [5]:
# split the data
train_size = int(0.7 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

In [6]:
len(train_dataset)

9823

In [7]:
len(test_dataset)

4211

In [8]:
# DataLoader class
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True, num_workers = 2, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size = 32)

In [9]:
# define NN class
class MyModel(nn.Module):
    def __init__(self, input_shape):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),

            nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        # flatten input size
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_shape)
            conv_output = self.conv(dummy_input)
            self.flattened_size = conv_output.view(1, -1).size(1)
            
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flattened_size, 128),
            nn.ReLU(),
            nn.Dropout(p = 0.3),
            
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p = 0.3),

            nn.Linear(64, 6)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.classifier(x)
        return x

In [10]:
# define parameters
learning_rate = 0.001
epochs = 100
input_shape = (3, 150, 150)

In [11]:
# define Model, loss and optimizer
model = MyModel(input_shape).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [12]:
for images, labels in train_loader:
    print(labels)
    break

tensor([3, 3, 4, 0, 0, 1, 5, 2, 4, 4, 3, 1, 1, 1, 3, 1, 0, 1, 0, 4, 0, 4, 1, 1,
        3, 3, 5, 4, 0, 0, 4, 3])


In [13]:
# define training loop
for epoch in range(epochs):
    total_epochs_loss = 0
    for batch_features, batch_labels in train_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_epochs_loss += loss.item()
    avg_loss = total_epochs_loss / len(train_loader)
    print(f'Epoch: {epoch + 1}, Loss: {avg_loss}')

Epoch: 1, Loss: 1.6844995988696716
Epoch: 2, Loss: 1.1916416330135606
Epoch: 3, Loss: 1.0772870025727959
Epoch: 4, Loss: 0.9656938729923012
Epoch: 5, Loss: 0.9087995547036783
Epoch: 6, Loss: 0.8561694869583515
Epoch: 7, Loss: 0.8239715869147148
Epoch: 8, Loss: 0.7819599937344218
Epoch: 9, Loss: 0.7615859069731026
Epoch: 10, Loss: 0.7352765593738432
Epoch: 11, Loss: 0.6990015638767703
Epoch: 12, Loss: 0.6886131971200825
Epoch: 13, Loss: 0.6691558592094272
Epoch: 14, Loss: 0.6408232373795214
Epoch: 15, Loss: 0.6098216103711424
Epoch: 16, Loss: 0.5802394204108645
Epoch: 17, Loss: 0.5705444890435433
Epoch: 18, Loss: 0.5763610534442752
Epoch: 19, Loss: 0.5547906135600242
Epoch: 20, Loss: 0.5415531470643581
Epoch: 21, Loss: 0.5242709280128199
Epoch: 22, Loss: 0.5119236456745999
Epoch: 23, Loss: 0.5144234291893651
Epoch: 24, Loss: 0.4937736757512201
Epoch: 25, Loss: 0.4959229774020782
Epoch: 26, Loss: 0.47769072433442167
Epoch: 27, Loss: 0.5057812027317693
Epoch: 28, Loss: 0.46052819563239716

In [14]:
# eval mode
model.eval()

MyModel(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=87616, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.3, inplace=False)
    (7): Linear(in_features=64, out_features=6, bias=True)
  )
)

In [16]:
# evaluation
total = 0
correct = 0
with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)
        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        total += batch_labels.shape[0]
        correct += (predicted == batch_labels).sum().item()
acc = correct/total
print(f'Accuracy: {acc}')

Accuracy: 0.7193065780099739
